# Data analysis and exploration

We will explore the data and see 

In [9]:
import matplotlib.pyplot as plt
import plotly.offline as py
# import plotly.graph_objs as go

import glob
import numpy  as np
import pandas as pd
import datetime

%matplotlib inline
py.init_notebook_mode(connected=True)

In [31]:
delay_data = pd.DataFrame()
date_start = '2017-01-01'
date_end   = '2017-04-23'

for f in glob.glob('./data/*.csv'):
    name =  f.replace('.traffic.risc-software.at.csv','').replace('./data/cur.','')
    _data_ = pd.read_csv(f,  parse_dates = [0], index_col = 0)
    delay_data[name.replace('.','_')] = _data_[date_start:date_end].delay

In [37]:
import plotly.offline as py
import plotly.graph_objs as go 

from datetime import datetime

df = delay_data

pl_data = []
for col in df.columns :
    trace = go.Scatter(x=df.index,
                       y=df[col].values ,
                       name=col,
                       visible=False,
                      )

    pl_data.append(trace)
    
# high_annotations=[dict(x='2016-03-01',
#                        y=df.High.mean(),
#                        xref='x', yref='y',
#                        text='High Average:<br>'+str(df.High.mean()),
#                        ax=0, ay=-40),
#                   dict(x=df.High.idxmax(),
#                        y=df.High.max(),
#                        xref='x', yref='y',
#                        text='High Max:<br>'+str(df.High.max()),
#                        ax=0, ay=-40)]
# low_annotations=[dict(x='2015-05-01',
#                       y=df.Low.mean(),
#                       xref='x', yref='y',
#                       text='Low Average:<br>'+str(df.Low.mean()),
#                       ax=0, ay=40),
#                  dict(x=df.High.idxmin(),
#                       y=df.Low.min(),
#                       xref='x', yref='y',
#                       text='Low Min:<br>'+str(df.Low.min()),
#                       ax=0, ay=40)]

buttons = []
for col in df.columns :
    button = dict(label = 'High',
                  method = 'update',
                  args = [{'visible': [True, True, False, False]},
                          {'title': col }])
    buttons.append(trace)

    
updatemenus = list([
    dict(type="buttons",
         active=-1,
         buttons=buttons
        )
])

# updatemenus = list([
#     dict(type="buttons",
#          active=-1,
#          buttons=list([   
#             dict(label = 'High',
#                  method = 'update',
#                  args = [{'visible': [True, True, False, False]},
#                          {'title': 'Yahoo High',
#                           'annotations': high_annotations}]),
#             dict(label = 'Low',
#                  method = 'update',
#                  args = [{'visible': [False, False, True, True]},
#                          {'title': 'Yahoo Low',
#                           'annotations': low_annotations}]),
#             dict(label = 'Both',
#                  method = 'update',
#                  args = [{'visible': [True, True, True, True]},
#                          {'title': 'Yahoo',
#                           'annotations': high_annotations+low_annotations}]),
#             dict(label = 'Reset',
#                  method = 'update',
#                  args = [{'visible': [True, False, True, False]},
#                          {'title': 'Yahoo',
#                           'annotations': []}])
#         ]),
#     )
# ])

layout = dict(title='Yahoo', showlegend=False,
              updatemenus=updatemenus)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='update_button')


PlotlyError: Invalid 'figure_or_data' argument. Plotly will not be able to properly parse the resulting JSON. If you want to send this 'figure_or_data' to Plotly anyway (not recommended), you can set 'validate=False' as a plot option.
Here's why you're seeing this error:

'y' is not allowed in 'button'

Path To Error: ['layout']['updatemenus'][0]['buttons'][0]['y']

Valid attributes for 'button' at path ['layout']['updatemenus'][0]['buttons'][0] under parents ['figure', 'layout', 'updatemenus', 'updatemenu', 'buttons']:

    ['args', 'label', 'method']

Run `<button-object>.help('attribute')` on any of the above.
'<button-object>' is the object at ['layout']['updatemenus'][0]['buttons'][0]

In [38]:
from datetime import datetime
import pandas_datareader.data as web

df = web.DataReader("aapl", 'yahoo',
                    datetime(2015, 1, 1),
                    datetime(2016, 7, 1))

trace_high = go.Scatter(x=df.index,
                        y=df.High,
                        name='High',
                        line=dict(color='#33CFA5'))

trace_high_avg = go.Scatter(x=df.index,
                            y=[df.High.mean()]*len(df.index),
                            name='High Average',
                            visible=False,
                            line=dict(color='#33CFA5', dash='dash'))

trace_low = go.Scatter(x=df.index,
                       y=df.Low,
                       name='Low',
                       line=dict(color='#F06A6A'))

trace_low_avg = go.Scatter(x=df.index,
                           y=[df.Low.mean()]*len(df.index),
                           name='Low Average',
                           visible=False,
                           line=dict(color='#F06A6A', dash='dash'))

data = [trace_high, trace_high_avg, trace_low, trace_low_avg]

high_annotations=[dict(x='2016-03-01',
                       y=df.High.mean(),
                       xref='x', yref='y',
                       text='High Average:<br>'+str(df.High.mean()),
                       ax=0, ay=-40),
                  dict(x=df.High.idxmax(),
                       y=df.High.max(),
                       xref='x', yref='y',
                       text='High Max:<br>'+str(df.High.max()),
                       ax=0, ay=-40)]
low_annotations=[dict(x='2015-05-01',
                      y=df.Low.mean(),
                      xref='x', yref='y',
                      text='Low Average:<br>'+str(df.Low.mean()),
                      ax=0, ay=40),
                 dict(x=df.High.idxmin(),
                      y=df.Low.min(),
                      xref='x', yref='y',
                      text='Low Min:<br>'+str(df.Low.min()),
                      ax=0, ay=40)]

updatemenus = list([
    dict(type="buttons",
         active=-1,
         buttons=list([   
            dict(label = 'High',
                 method = 'update',
                 args = [{'visible': [True, True, False, False]},
                         {'title': 'Yahoo High',
                          'annotations': high_annotations}]),
            dict(label = 'Low',
                 method = 'update',
                 args = [{'visible': [False, False, True, True]},
                         {'title': 'Yahoo Low',
                          'annotations': low_annotations}]),
            dict(label = 'Both',
                 method = 'update',
                 args = [{'visible': [True, True, True, True]},
                         {'title': 'Yahoo',
                          'annotations': high_annotations+low_annotations}]),
            dict(label = 'Reset',
                 method = 'update',
                 args = [{'visible': [True, False, True, False]},
                         {'title': 'Yahoo',
                          'annotations': []}])
        ]),
    )
])

layout = dict(title='Yahoo', showlegend=False,
              updatemenus=updatemenus)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='update_button')